In [9]:
import os
import sys
path = !echo ${VIRTUAL_ENV}
path = os.path.join(path[0], '..')
sys.path.append(path)
from helper.utils import load_from_pickle, load_from_shelve
import numpy as np
import pandas as pd
import collections
# from plotly import tools
# import plotly as py
# import plotly.graph_objs as go
#import impywidgets as widget
# py.offline.init_notebook_mode(connected=True)

In [14]:
base_data_path = "/media/jackalhan/ext_ssd"

In [15]:
debug_files = {}
for i, structure in enumerate(os.walk(base_data_path)):
    root, dirs, files = structure
    for file in files:
        if file.endswith(".pkl"):            
            debug_files[i] = os.path.join(root, file)
["{} --> {}".format(key, value) for key, value in debug_files.items()]

['0 --> /media/jackalhan/ext_ssd/debug_dict.pkl',
 '16 --> /media/jackalhan/ext_ssd/Rethinking_Running_Data/dev/tfidf/source_destination_tfidf_embeddings_token_verbose_None.pkl',
 '3 --> /media/jackalhan/ext_ssd/Rethinking_Running_Data/dev/glove/source_destination_glove_embeddings_token_verbose_None.pkl',
 '35 --> /media/jackalhan/ext_ssd/Rethinking_Running_Data/dev/bert/bert_new_destination_tokens.pkl',
 '48 --> /media/jackalhan/ext_ssd/Rethinking_Running_Data/train/bert/bert_new_destination_tokens.pkl']

In [16]:
model_indx =0
my_dict = load_from_pickle(os.path.join(base_data_path, 
                                        debug_files[model_indx]))
num_of_epochs = my_dict['epochs']

In [17]:
eval_metrics = ['recall', 'precision', 'map', 'dcg', 'ndcg', 'arp', 'mrp']
top_k = ['eval_metric_name', '1', '2', '3', '5', '10', '20', '50', 'all']

#extract feature names and k into the hiarchical format
eval_metrics = {}
for key, value in my_dict['1'].items():
    _ = key.split('_')
    fn_name = _[0]
    k = _[1]
    if fn_name not in eval_metrics:
        eval_metrics[fn_name]={}
    if k == 'all':
        eval_metrics[fn_name][1000] = value
    else:
        eval_metrics[fn_name][int(k)] = value

# now place them to the dataframes
dataset = []
for fn_name, ks in eval_metrics.items():
    ks_ = collections.OrderedDict(sorted(ks.items()))
    row = []
    for k in top_k:
        if k == 'eval_metric_name':
            row.append(fn_name)
            continue
        if k == 'all':
            _k = 1000
        else:
            _k = int(k)
        if _k in ks_:
            row.append(ks[int(_k)])
        else:
            row.append('-')
    dataset.append(row)
    #print(fn_name)
df_dataset = pd.DataFrame(data=dataset, columns=top_k)
df_dataset

,eval_metric_name,1,2,3,5,10,20,50,all
0,arp,-,-,-,-,-,-,-,102.108
1,dcg,0.296878,0.357165,0.386541,0.414938,0.445104,0.466758,0.48464,-
2,map,0.296878,0.344655,0.364238,0.379991,0.392568,0.398554,0.401482,-
3,ndcg,0.296878,0.357165,0.386541,0.414938,0.445104,0.466758,0.48464,-
4,precision,0.296878,0.196216,0.150394,0.10403,0.0612772,0.0349054,0.0157502,-
5,mrp,-,-,-,-,-,-,-,0.402797
6,recall,0.296878,0.392431,0.451183,0.520151,0.612772,0.698108,0.787512,-
